In [1]:
import pandas as pd

# 1. Read in files

### Reading in raw files
Read in files from paths, defining the expected schema to be read

In [2]:
# Load the CSV file into a Pandas DataFrame
stocks_path = '../raw_data/top100_stocks_dataset.csv'
etf_path = '../raw_data/etf_concat.csv'
stocks_info_path = '../raw_data/stock_info.csv'
stocks_financials_path = '../raw_data/stock_financials.csv'
stocks_news_path = '../raw_data/stock_news.csv'
stocks_tweets_path = '../raw_data/stock_tweets.csv'

security_prices_schema = {
    'Ticker': str,
    'Date': object,
    'Open': float,
    'Close': float,
    'High': float,
    'Low': float,
    'AdjClose': float,
    'Volume': 'Int64'
}

stocks_info_schema = {
    "ticker" : str,
    "short name" : str,
    "industry" : str,
    "description" : str,
    "website" : str,
    "ceo" : str,
    "exchange" : str,
    "market cap" : str
}

stocks_financials_schema = {
    "period" : str,
    "tickers" : str,
    "indicator" : str,
    "unit" : str,
    "amount" : str
}

stocks_news_schema = {
    "title" : str,
    "date" : object,
    "stock" : str
}

stocks_tweets_schema = {
    "ID" : int,
    "Date" : object,
    "Text" : str,
    "Likes" : int,
    "Retweets" : int,
    "Replies" : int,
    "Ticker" : str
}

df_stocks_raw = pd.read_csv(stocks_path, dtype=security_prices_schema, parse_dates=['Date'])
df_etf_raw = pd.read_csv(etf_path, dtype=security_prices_schema, parse_dates=['Date'])
df_stock_info_raw = pd.read_csv(stocks_info_path, dtype=stocks_info_schema, usecols=list(stocks_info_schema.keys()))
df_stock_financials_raw = pd.read_csv(stocks_financials_path, dtype=stocks_financials_schema)
df_stock_news_raw = pd.read_csv(stocks_news_path, dtype=stocks_news_schema, parse_dates=['date'], index_col=0)
df_stock_tweets_raw = pd.read_csv(stocks_tweets_path, dtype=stocks_tweets_schema, parse_dates=['Date'], index_col=0)

C:\Users\Kenzo\AppData\Local\Temp\ipykernel_29144\1397311771.py:60: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_stock_tweets_raw = pd.read_csv(stocks_tweets_path, dtype=stocks_tweets_schema, parse_dates=['Date'], index_col=0)


### Securities Tables
Filter the raw stock and ETF prices dataset: drop NaN, parse dates, and fix column names to match schema.
Then, concatenate them to make the base for the securities / security prices tables.

In [3]:
df_stocks_filtered = df_stocks_raw.dropna(axis=0).copy().reset_index(drop=True)
df_stocks_filtered['Date'] = df_stocks_filtered['Date'].dt.strftime('%Y-%m-%d')
df_stocks_filtered.rename(columns={"Adj Close" : "AdjClose"}, inplace=True)
df_stocks_filtered

,Date,Open,High,Low,Close,AdjClose,Volume,Ticker
0,1980-12-12,0.513393,0.515625,0.513393,0.513393,0.406782,117258400,AAPL
1,1980-12-15,0.488839,0.488839,0.486607,0.486607,0.385558,43971200,AAPL
2,1980-12-16,0.453125,0.453125,0.450893,0.450893,0.357260,26432000,AAPL
3,1980-12-17,0.462054,0.464286,0.462054,0.462054,0.366103,21610400,AAPL
4,1980-12-18,0.475446,0.477679,0.475446,0.475446,0.376715,18362400,AAPL
...,...,...,...,...,...,...,...,...
541912,2020-03-26,60.290001,62.000000,59.439999,60.549999,60.549999,3618400,ZS
541913,2020-03-27,59.540001,60.840000,57.939999,58.630001,58.630001,3422300,ZS
541914,2020-03-30,59.160000,60.500000,57.770000,58.840000,58.840000,3815100,ZS
541915,2020-03-31,58.500000,62.330002,58.290001,60.860001,60.860001,3648000,ZS


In [4]:
df_etf_filtered = df_etf_raw.dropna(axis=0).copy().reset_index(drop=True)
df_etf_filtered['Date'] = df_etf_filtered['Date'].dt.strftime('%Y-%m-%d')
df_etf_filtered.rename(columns={"Adj Close" : "AdjClose"}, inplace=True)
df_etf_filtered

,Date,Open,High,Low,Close,AdjClose,Volume,Ticker
0,2018-08-15,11.840000,11.840000,11.740000,11.740000,11.740000,27300,AAAU
1,2018-08-16,11.780000,11.800000,11.740000,11.740000,11.740000,428400,AAAU
2,2018-08-17,11.800000,11.820000,11.770000,11.820000,11.820000,52400,AAAU
3,2018-08-20,11.880000,11.910000,11.850000,11.900000,11.900000,28700,AAAU
4,2018-08-21,11.920000,11.950000,11.890000,11.930000,11.930000,30600,AAAU
...,...,...,...,...,...,...,...,...
3954277,2020-03-26,54.099998,56.790001,54.099998,56.720001,56.720001,9100,_PRN
3954278,2020-03-27,54.360001,55.900002,54.360001,54.889999,54.889999,8400,_PRN
3954279,2020-03-30,55.509998,56.529999,55.509998,56.529999,56.529999,11700,_PRN
3954280,2020-03-31,55.759998,56.610001,55.490002,55.660000,55.660000,5400,_PRN


In [5]:
df_securities = pd.concat([df_stocks_filtered, df_etf_filtered]).reset_index(drop=True)
df_securities

,Date,Open,High,Low,Close,AdjClose,Volume,Ticker
0,1980-12-12,0.513393,0.515625,0.513393,0.513393,0.406782,117258400,AAPL
1,1980-12-15,0.488839,0.488839,0.486607,0.486607,0.385558,43971200,AAPL
2,1980-12-16,0.453125,0.453125,0.450893,0.450893,0.357260,26432000,AAPL
3,1980-12-17,0.462054,0.464286,0.462054,0.462054,0.366103,21610400,AAPL
4,1980-12-18,0.475446,0.477679,0.475446,0.475446,0.376715,18362400,AAPL
...,...,...,...,...,...,...,...,...
4496194,2020-03-26,54.099998,56.790001,54.099998,56.720001,56.720001,9100,_PRN
4496195,2020-03-27,54.360001,55.900002,54.360001,54.889999,54.889999,8400,_PRN
4496196,2020-03-30,55.509998,56.529999,55.509998,56.529999,56.529999,11700,_PRN
4496197,2020-03-31,55.759998,56.610001,55.490002,55.660000,55.660000,5400,_PRN


### Stock/ETF Information Table
Filter the raw stock info dataset: fix column names to match schema, drop rows with no Ticker.

In [6]:
df_stock_info_filtered = df_stock_info_raw.copy().reset_index(drop=True)
df_stock_info_filtered.columns = df_stock_info_filtered.columns.str.capitalize()
df_stock_info_filtered.rename(columns={"Short name" : "Name", "Ceo" : "CEO", "Market cap" : "MarketCap"}, inplace=True)
df_stock_info_filtered = df_stock_info_filtered.dropna(subset=['Ticker'])
df_stock_info_filtered

,Ticker,Name,Industry,Description,Website,CEO,Exchange,MarketCap
0,A,Agilent,Medical Diagnostics & Research,Agilent Technologies Inc is engaged in life sc...,http://www.agilent.com,Michael R. McMullen,New York Stock Exchange,24218068096
1,AA,Alcoa,Metals & Mining,Alcoa Corp is an integrated aluminum company. ...,http://www.alcoa.com,Roy Christopher Harvey,New York Stock Exchange,5374966543
2,AABA,Altaba,Asset Management,"Altaba Inc is an independent, non-diversified,...",http://www.altaba.com,Thomas J. Mcinerney,Nasdaq Global Select,41223683414
3,AAC,AAC,Health Care Providers,AAC Holdings Inc provides inpatient and outpat...,http://www.americanaddictioncenters.org,Michael T. Cartwright,New York Stock Exchange,63720099
4,AADR,AdvisorShares Dorsey Wright,NaN,The investment seeks long-term capital appreci...,http://www.advisorshares.com,NaN,NYSE Arca,103161180
...,...,...,...,...,...,...,...,...
6363,ZTS,Zoetis,Drug Manufacturers,Zoetis Inc is a developer and manufacturer of ...,http://www.zoetis.com,Juan Ramon Alaix,New York Stock Exchange,42056267956
6364,ZUMZ,Zumiez,Retail - Apparel & Specialty,Zumiez Inc is a multi-channel specialty retail...,http://www.zumiez.com,Richard M. Brooks,Nasdaq Global Select,615036827
6365,ZUO,Zuora,Application Software,Zuora Inc provides cloud-based software on a s...,https://www.zuora.com,Tien Tzuo,New York Stock Exchange,2304595224
6366,ZYME,Zymeworks,Biotechnology,Zymeworks Inc is a clinical-stage biopharmaceu...,http://www.zymeworks.com,Ali Tehrani,New York Stock Exchange,504287824


### Stock Financials Table
Filter the raw stock financials dataset: drop NaN, parse period into year and quarter, fix column names to match schema, and fix datatypes.

In [7]:
df_stock_financials_filtered = df_stock_financials_raw.dropna(axis=0).copy().reset_index(drop=True)
df_stock_financials_filtered['Quarter'] = df_stock_financials_filtered['period'].apply(lambda s: s[-2:])
df_stock_financials_filtered['Year'] = df_stock_financials_filtered['period'].apply(lambda s: s[:-3])
df_stock_financials_filtered.drop(columns=['period'], inplace=True)
df_stock_financials_filtered.columns = df_stock_financials_filtered.columns.str.capitalize()
df_stock_financials_filtered['Amount'] = df_stock_financials_filtered['Amount'].str.replace(',', '').astype('Int64')
df_stock_financials_filtered.rename(columns={"Tickers" : "Ticker"}, inplace=True)
df_stock_financials_filtered

,Company,Ticker,Indicator,Unit,Amount,Quarter,Year
0,"1347 Property Insurance Holdings, Inc.",PIH,Assets,US Dollar,42854000,Q1,2014
1,"1347 Property Insurance Holdings, Inc.",PIH,"Cash and Cash Equivalents, at Carrying Value",US Dollar,18330000,Q1,2014
2,"1347 Property Insurance Holdings, Inc.",PIH,"Cash and Cash Equivalents, Period Increase (De...",US Dollar,3323000,Q1,2014
3,"1347 Property Insurance Holdings, Inc.",PIH,Final Revenue,US Dollar,4173000,Q1,2014
4,"1347 Property Insurance Holdings, Inc.",PIH,Income from Continuing Operations before Taxes,US Dollar,2307000,Q1,2014
...,...,...,...,...,...,...,...
186331,ZYNGA INC,ZNGA,Final Revenue,US Dollar,190540000,Q4,2016
186332,ZYNGA INC,ZNGA,Net Income (Loss),US Dollar,-35432000,Q4,2016
186333,ZYNGA INC,ZNGA,Operating Income (Loss),US Dollar,-34461000,Q4,2016
186334,ZYNGA INC,ZNGA,Total Equity,US Dollar,1580664000,Q4,2016


However, we notice that some tickers are in a different format. For instance see `Google`'s ticker below.
This happens because there can be multiple classes of the same stock.
To fix this, we will split these cases into their own rows, that is, we will consider each
class as its own ticker. In the case below, we would have two rows for each, one for
`GOOG` and another for `GOOGL`.

In [8]:
df_stock_financials_filtered[df_stock_financials_filtered['Ticker'].isin([', GOOG, GOOGL'])]

,Company,Ticker,Indicator,Unit,Amount,Quarter,Year
7787,Alphabet Inc.,", GOOG, GOOGL","Cash and Cash Equivalents, at Carrying Value",US Dollar,15605000000,Q3,2014
7788,Alphabet Inc.,", GOOG, GOOGL",Final Revenue,US Dollar,16523000000,Q3,2014
7789,Alphabet Inc.,", GOOG, GOOGL",Income from Continuing Operations before Taxes,US Dollar,3857000000,Q3,2014
7790,Alphabet Inc.,", GOOG, GOOGL",Net Income (Loss),US Dollar,2739000000,Q3,2014
7791,Alphabet Inc.,", GOOG, GOOGL",Operating Income (Loss),US Dollar,3724000000,Q3,2014
...,...,...,...,...,...,...,...
74838,GOOGLE INC.,", GOOG, GOOGL",Income from Continuing Operations before Taxes,US Dollar,4891000000,Q3,2015
74839,GOOGLE INC.,", GOOG, GOOGL",Net Income (Loss),US Dollar,3979000000,Q3,2015
74840,GOOGLE INC.,", GOOG, GOOGL",Operating Income (Loss),US Dollar,4708000000,Q3,2015
74841,GOOGLE INC.,", GOOG, GOOGL",Total Equity,US Dollar,116241000000,Q3,2015


In [9]:
df_split = df_stock_financials_filtered['Ticker'].str.split(', ', expand=True).stack().reset_index(level=1, drop=True).to_frame('Ticker')
df_stock_financials_filtered = df_stock_financials_filtered.drop('Ticker', axis=1).merge(df_split, left_index=True, right_index=True)

df_stock_financials_filtered.reset_index(drop=True, inplace=True)
df_stock_financials_filtered

,Company,Indicator,Unit,Amount,Quarter,Year,Ticker
0,"1347 Property Insurance Holdings, Inc.",Assets,US Dollar,42854000,Q1,2014,PIH
1,"1347 Property Insurance Holdings, Inc.","Cash and Cash Equivalents, at Carrying Value",US Dollar,18330000,Q1,2014,PIH
2,"1347 Property Insurance Holdings, Inc.","Cash and Cash Equivalents, Period Increase (De...",US Dollar,3323000,Q1,2014,PIH
3,"1347 Property Insurance Holdings, Inc.",Final Revenue,US Dollar,4173000,Q1,2014,PIH
4,"1347 Property Insurance Holdings, Inc.",Income from Continuing Operations before Taxes,US Dollar,2307000,Q1,2014,PIH
...,...,...,...,...,...,...,...
212839,ZYNGA INC,Final Revenue,US Dollar,190540000,Q4,2016,ZNGA
212840,ZYNGA INC,Net Income (Loss),US Dollar,-35432000,Q4,2016,ZNGA
212841,ZYNGA INC,Operating Income (Loss),US Dollar,-34461000,Q4,2016,ZNGA
212842,ZYNGA INC,Total Equity,US Dollar,1580664000,Q4,2016,ZNGA


In [10]:
df_stock_financials_filtered[df_stock_financials_filtered['Ticker'].isin(['GOOGL', 'GOOG'])]

,Company,Indicator,Unit,Amount,Quarter,Year,Ticker
8535,Alphabet Inc.,"Cash and Cash Equivalents, at Carrying Value",US Dollar,15605000000,Q3,2014,GOOG
8536,Alphabet Inc.,"Cash and Cash Equivalents, at Carrying Value",US Dollar,15605000000,Q3,2014,GOOGL
8538,Alphabet Inc.,Final Revenue,US Dollar,16523000000,Q3,2014,GOOG
8539,Alphabet Inc.,Final Revenue,US Dollar,16523000000,Q3,2014,GOOGL
8541,Alphabet Inc.,Income from Continuing Operations before Taxes,US Dollar,3857000000,Q3,2014,GOOG
...,...,...,...,...,...,...,...
84872,GOOGLE INC.,Operating Income (Loss),US Dollar,4708000000,Q3,2015,GOOGL
84874,GOOGLE INC.,Total Equity,US Dollar,116241000000,Q3,2015,GOOG
84875,GOOGLE INC.,Total Equity,US Dollar,116241000000,Q3,2015,GOOGL
84877,GOOGLE INC.,Total Liabilities and Equity,US Dollar,144281000000,Q3,2015,GOOG


We see above that we now have rows for each `GOOG` and `GOOGL`, as we wanted.

### Financial News Table
Filter the raw stock news dataset: drop NaN, parse dates, and fix column names to match schema.

In [11]:
df_stock_news_filtered = df_stock_news_raw.dropna(axis=0).copy().reset_index(drop=True)
df_stock_news_filtered['Index'] = df_stock_news_filtered.index
df_stock_news_filtered['date'] = pd.to_datetime(df_stock_news_filtered['date'], utc=True)
df_stock_news_filtered['date'] = df_stock_news_filtered['date'].dt.strftime('%Y-%m-%d')
df_stock_news_filtered.columns = df_stock_news_filtered.columns.str.capitalize()
df_stock_news_filtered.rename(columns={"Title" : "Headline", "Stock" : "Ticker"}, inplace=True)
df_stock_news_filtered

,Headline,Date,Ticker,Index
0,Stocks That Hit 52-Week Highs On Friday,2020-06-05,A,0
1,Stocks That Hit 52-Week Highs On Wednesday,2020-06-03,A,1
2,71 Biggest Movers From Friday,2020-05-26,A,2
3,46 Stocks Moving In Friday's Mid-Day Session,2020-05-22,A,3
4,B of A Securities Maintains Neutral on Agilent...,2020-05-22,A,4
...,...,...,...,...
1397886,Top Narrow Based Indexes For August 29,2011-08-29,ZX,1397886
1397887,Recap: Wednesday's Top Percentage Gainers and ...,2011-06-22,ZX,1397887
1397888,UPDATE: Oppenheimer Color on China Zenix Auto ...,2011-06-21,ZX,1397888
1397889,Oppenheimer Initiates China Zenix At Outperfor...,2011-06-21,ZX,1397889


### Financial Tweets Table
Filter the raw financial tweets dataset: drop NaN, parse dates, and fix column names to match schema.

In [12]:
df_stock_tweets_filtered = df_stock_tweets_raw.dropna(axis=0).copy().reset_index(drop=True)
df_stock_tweets_filtered.columns = df_stock_tweets_filtered.columns.str.capitalize()
df_stock_tweets_filtered['Date'] = pd.to_datetime(df_stock_tweets_filtered['Date'].astype(float), unit='s')
df_stock_tweets_filtered['Date'] = df_stock_tweets_filtered['Date'].dt.strftime('%Y-%m-%d')
df_stock_tweets_filtered.rename(columns={"Id" : "ID"}, inplace=True)
df_stock_tweets_filtered

,ID,Date,Text,Likes,Retweets,Replies,Quotes,Ticker
0,668610561,2022-12-30,Few of mine to be thankful for in 2022: $IBM ...,208,7,25,1,DE
1,-2120876031,2022-12-31,Bill Gates Top 10 Dividend Stocks💰 \n\n$DE 🚜 1...,46,12,2,0,DE
2,-589680636,2022-12-31,"2022 was a ""stay-away"" market, as @chasharris1...",38,4,2,2,DE
3,958005248,2022-12-30,Checking out my Dividend Growth Pie and turns ...,19,0,3,0,DE
4,-1902477311,2022-12-31,Great list by @thedividendclub on IG\n\n$SCHW ...,83,15,2,1,DE
...,...,...,...,...,...,...,...,...
104504,789954561,2018-01-31,Here's how @JimCramer wants you to look at $AA...,106,30,6,3,AAPL
104505,1398190080,2018-02-03,"With incredible technology like this, can’t be...",98,7,3,3,AAPL
104506,-1143500799,2018-02-02,how are the myriad brokers and analysts who to...,409,41,69,11,AAPL
104507,966168576,2018-01-27,#earnings for the week\n\n$AAPL $BABA $FB $AMZ...,286,223,10,53,AAPL


In [13]:
set(df_stock_tweets_raw['Ticker'])

{'AAPL', 'DE', 'ILMN', 'PFE', 'TSLA', 'VEGI'}

As we can see above, unfortunately the tweets dataset only contains tweets
about 6 companies. This would not be very helpful in the project, since it would
limit the feature to only a few companies. Therefore, we chose to drop the tweets
feature.

# 2. Find Ticker Intersection
Now that we have the basic filtered tables, we need to ensure consistency of tickers across tables.
That is, we have to ensure that the foreign keys will be valid when referencing the Ticker in
the securities table. We just have to be careful not to distinguish between stocks and ETFs
to not drop more than we need to.

To achieve this, we can find the intersection of the ticker values for all tables
and use that as our reference. Then, we drop any rows that reference a ticker
that is not in this master intersection.

We know that the `df_securities` dataframe has both stocks and ETFs, combined from `df_stocks_filtered`
and `df_etf_filtered`, and `df_stock_info_filtered` also should contain both stocks and ETFs.
So we find their intersection to ensure that all securities will have their info
available.

In [14]:

print("Calculating intersection of Stocks...")
print(f"\tLength of original filtered stocks: {len(set(df_stocks_filtered['Ticker']))}")
stocks_tickers = pd.Series(list(set(df_stocks_filtered['Ticker']) & set(df_stock_news_filtered['Ticker'])))
print(f"\tLength of stocks after intersection with stock news: {len(stocks_tickers)}")
stocks_tickers = pd.Series(list(set(stocks_tickers) & set(df_stock_financials_filtered['Ticker'])))
print(f"\tLength of stocks after intersection with financials: {len(stocks_tickers)}")
stocks_tickers = pd.Series(list(set(stocks_tickers) & set(df_stock_info_filtered['Ticker'])))
print(f"\tLength of stocks after intersection with stock information: {len(stocks_tickers)}")
print(f"Final length of stocks after intersections: {len(stocks_tickers)}")

print("\n\nCalculating intersection of ETFs...")
print(f"\tLength of original filtered ETFs: {len(set(df_etf_filtered['Ticker']))}")
etf_tickers = pd.Series(list(set(df_etf_filtered['Ticker']) & set(df_stock_info_filtered['Ticker'])))
print(f"\tLength of ETFs after intersection with ETF information: {len(etf_tickers)}")
print(f"Final length of ETFs after intersections: {len(etf_tickers)}")

Calculating intersection of Stocks...
	Length of original filtered stocks: 93
	Length of stocks after intersection with stock news: 71
	Length of stocks after intersection with financials: 51
	Length of stocks after intersection with stock information: 51
Final length of stocks after intersections: 51


Calculating intersection of ETFs...
	Length of original filtered ETFs: 2165
	Length of ETFs after intersection with ETF information: 825
Final length of ETFs after intersections: 825


# 3. Filtering out tickers

Now, with the intersection of ETFs and stocks separately, we can use these series
to find what our final tables should be. We do so by removing the rows in the
filtered dataframes with tickers that are not in `stock_tickers` or `etf_tickers`.

### Securities
Since they should include both stocks and ETFs, we check if they are in either of them.

### Securities
Since they should include both stocks and ETFs, we check if they are in either of them.

In [15]:
print(f"\tLength of original securities tickers: {len(set(df_securities['Ticker']))}")
df_security_prices_final = df_securities[
    df_securities['Ticker'].isin(stocks_tickers) |
    df_securities['Ticker'].isin(etf_tickers)
].reset_index(drop=True)
print(f"\tLength of final securities tickers: {len(set(df_security_prices_final['Ticker']))}")

# Reorder columns to fit schema
df_security_prices_final = df_security_prices_final[['Ticker', 'Date', 'Open', 'High', 'Low', 'Close', 'AdjClose', 'Volume']]
df_security_prices_final

	Length of original securities tickers: 2258
	Length of final securities tickers: 876


,Ticker,Date,Open,High,Low,Close,AdjClose,Volume
0,AAPL,1980-12-12,0.513393,0.515625,0.513393,0.513393,0.406782,117258400
1,AAPL,1980-12-15,0.488839,0.488839,0.486607,0.486607,0.385558,43971200
2,AAPL,1980-12-16,0.453125,0.453125,0.450893,0.450893,0.357260,26432000
3,AAPL,1980-12-17,0.462054,0.464286,0.462054,0.462054,0.366103,21610400
4,AAPL,1980-12-18,0.475446,0.477679,0.475446,0.475446,0.376715,18362400
...,...,...,...,...,...,...,...,...
2936695,ZSL,2020-03-26,33.549999,34.580002,32.790001,34.200001,34.200001,48300
2936696,ZSL,2020-03-27,34.419998,35.389999,34.330002,34.630001,34.630001,47400
2936697,ZSL,2020-03-30,36.599998,37.340000,35.410000,36.110001,36.110001,55200
2936698,ZSL,2020-03-31,36.259998,37.000000,34.869999,36.660000,36.660000,30400


Now, we create the `securities` dataframe, which should be of schema `(Ticker, Name, ETF)`, where `ETF` is a boolean
to indicate if it's an ETF or not.

In [16]:
# df_securities_final

# df_security_prices_final.join(df_stock_info_filtered, on='Ticker').groupby(by=['Ticker', 'Name'])
df_securities_final = pd.merge(
    df_security_prices_final,
    df_stock_info_filtered,
    on='Ticker')[['Ticker', 'Name']]\
    .groupby(['Ticker', 'Name'])\
    .agg(lambda x: x.iloc[0])\
    .reset_index(drop=True)

df_securities_final['ETF'] = df_securities_final['Ticker'].isin(etf_tickers).astype(int)
df_securities_final

,Ticker,Name,ETF
0,AADR,AdvisorShares Dorsey Wright,1
1,AAPL,Apple,0
2,AAXJ,iShares MSCI All Country Asia ex Japan Index Fund,1
3,ACWI,iShares MSCI ACWI Index Fund,1
4,ACWV,iShares Edge MSCI Min Vol Global,1
...,...,...,...
871,YCS,ProShares UltraShort Yen New,1
872,YINN,Direxion Daily FTSE China Bull 3x Shares,1
873,YXI,ProShares Short FTSE China 50,1
874,ZROZ,PIMCO 25 Year Zero Coupon U.S. Treasury Index ...,1


### Stock Info
Since they should include both stocks and ETFs, we check if they are in either of them.

In [17]:
print(f"\tLength of original stock info tickers: {len(set(df_stock_info_filtered['Ticker']))}")
df_stock_info_final = df_stock_info_filtered[
    df_stock_info_filtered['Ticker'].isin(stocks_tickers) |
    df_stock_info_filtered['Ticker'].isin(etf_tickers)
].reset_index(drop=True)
print(f"\tLength of final stock_info tickers: {len(set(df_stock_info_final['Ticker']))}")

# Reorder columns to fit schema
df_stock_info_final = df_stock_info_final[['Ticker', 'Name', 'Industry', 'Description', 'Website', 'CEO', 'Exchange', 'MarketCap']]
df_stock_info_final


	Length of original stock info tickers: 6368
	Length of final stock_info tickers: 876


,Ticker,Name,Industry,Description,Website,CEO,Exchange,MarketCap
0,AADR,AdvisorShares Dorsey Wright,NaN,The investment seeks long-term capital appreci...,http://www.advisorshares.com,NaN,NYSE Arca,103161180
1,AAPL,Apple,Computer Hardware,"Apple Inc is designs, manufactures and markets...",http://www.apple.com,Timothy D. Cook,Nasdaq Global Select,807491700000
2,AAXJ,iShares MSCI All Country Asia ex Japan Index Fund,NaN,The investment seeks to track the investment r...,http://www.ishares.com,NaN,NASDAQ Global Market,4541460000
3,ACWI,iShares MSCI ACWI Index Fund,NaN,The investment seeks to track the investment r...,http://www.ishares.com,NaN,NASDAQ Global Market,10432072000
4,ACWV,iShares Edge MSCI Min Vol Global,NaN,The investment seeks to track the investment r...,http://www.ishares.com,NaN,Cboe Global Markets EDGX,3825342000
...,...,...,...,...,...,...,...,...
871,YCS,ProShares UltraShort Yen New,NaN,The investment seeks to provide daily investme...,http://www.proshares.com,NaN,NYSE Arca,52229970
872,YINN,Direxion Daily FTSE China Bull 3x Shares,NaN,"The investment seeks daily investment results,...",http://www.direxioninvestments.com/,NaN,NYSE Arca,371221305
873,YXI,ProShares Short FTSE China 50,NaN,"The investment seeks daily investment results,...",http://www.proshares.com,NaN,NYSE Arca,6615000
874,ZROZ,PIMCO 25 Year Zero Coupon U.S. Treasury Index ...,NaN,The investment seeks to provide total return t...,http://www.pimcoetfs.com/fundinfo,NaN,NYSE Arca,165183900


### Stock Financials Info
Since they should only stocks, we check if they are in only `stock_tickers`.

In [18]:
print(f"\tLength of original stock financials tickers: {len(set(df_stock_financials_filtered['Ticker']))}")
df_stock_financials_final = df_stock_financials_filtered[
    df_stock_financials_filtered['Ticker'].isin(stocks_tickers)
].reset_index(drop=True)
print(f"\tLength of final stock_financials tickers: {len(set(df_stock_financials_final['Ticker']))}")

# Reorder columns to fit schema
df_stock_financials_final = df_stock_financials_final[['Ticker', 'Year', 'Quarter', 'Indicator', 'Unit', 'Amount',]]
df_stock_financials_final

	Length of original stock financials tickers: 2471
	Length of final stock_financials tickers: 51


,Ticker,Year,Quarter,Indicator,Unit,Amount
0,AMD,2014,Q1,Assets,US Dollar,4108000000
1,AMD,2014,Q1,"Cash and Cash Equivalents, at Carrying Value",US Dollar,554000000
2,AMD,2014,Q1,"Cash and Cash Equivalents, Period Increase (De...",US Dollar,-315000000
3,AMD,2014,Q1,Final Revenue,US Dollar,1397000000
4,AMD,2014,Q1,Gross Profit,US Dollar,487000000
...,...,...,...,...,...,...
5178,WYNN,2016,Q4,Final Revenue,US Dollar,1300433000
5179,WYNN,2016,Q4,Net Income (Loss),US Dollar,113800000
5180,WYNN,2016,Q4,Operating Income (Loss),US Dollar,138894000
5181,WYNN,2016,Q4,Total Equity,US Dollar,157949000


### Stock News
Since they should only stocks, we check if they are in only `stock_tickers`.

In [19]:
print(f"\tLength of original stock news tickers: {len(set(df_stock_news_filtered['Ticker']))}")
df_stock_news_final = df_stock_news_filtered[
    df_stock_news_filtered['Ticker'].isin(stocks_tickers)
].reset_index(drop=True)
df_stock_news_final['Index'] = df_stock_news_final.index # Update index
print(f"\tLength of final stock_news tickers: {len(set(df_stock_news_final['Ticker']))}")

# Reorder columns to fit schema
df_stock_news_final = df_stock_news_final[['Index', 'Headline', 'Date', 'Ticker',]]
df_stock_news_final

	Length of original stock news tickers: 6192
	Length of final stock_news tickers: 51


,Index,Headline,Date,Ticker
0,0,Tech Stocks And FAANGS Strong Again To Start D...,2020-06-10,AAPL
1,1,10 Biggest Price Target Changes For Wednesday,2020-06-10,AAPL
2,2,"Benzinga Pro's Top 5 Stocks To Watch For Wed.,...",2020-06-10,AAPL
3,3,"Deutsche Bank Maintains Buy on Apple, Raises P...",2020-06-10,AAPL
4,4,Apple To Let Users Trade In Their Mac Computer...,2020-06-10,AAPL
...,...,...,...,...
50164,50164,Nevada Gaming Control Board Reports Las Vegas ...,2018-01-31,WYNN
50165,50165,Morgan Stanley Maintains Overweight on Wynn Re...,2018-01-30,WYNN
50166,50166,Wynn Macau Says Formed Special Committee to In...,2018-01-30,WYNN
50167,50167,"ICYMI: AMD Earnings Preview, Dell And VMware, ...",2018-01-29,WYNN


# 4. Validation
To validate our process, we must ensure that all tickers are consistent. That is,
that no ticker will be missing some data that was expected. Moreover, we can check if
the columns expected to be primary keys are unique and non-null to ensure that it
can be properly read into our table.

### Securities

In [20]:
assert (df_securities_final['Ticker'].isin(stocks_tickers) | df_securities_final['Ticker'].isin(etf_tickers)).all()
print("Passed ticker validation")

assert not df_securities_final.isna().any().any()
print("Passed null validation")

assert not df_securities_final.duplicated(['Ticker']).any()
print("Passed uniqueness validation")

Passed ticker validation
Passed null validation
Passed uniqueness validation


### Security Prices

In [21]:
assert (df_security_prices_final['Ticker'].isin(stocks_tickers) | df_security_prices_final['Ticker'].isin(etf_tickers)).all()
print("Passed ticker validation")

assert not df_security_prices_final.isna().any().any()
print("Passed null validation")

assert not df_security_prices_final.duplicated(['Ticker', 'Date']).any()
print("Passed uniqueness validation")

Passed ticker validation
Passed null validation
Passed uniqueness validation


### Stock Info

In [22]:
assert (df_stock_info_final['Ticker'].isin(stocks_tickers) | df_stock_info_final['Ticker'].isin(etf_tickers)).all()
print("Passed ticker validation")

assert not df_stock_info_final[['Ticker', 'Name']].isna().any().any()
print("Passed null validation")

assert not df_stock_info_final.duplicated(['Ticker']).any()
print("Passed uniqueness validation")

Passed ticker validation
Passed null validation
Passed uniqueness validation


### Stock Financials

In [23]:
assert (df_stock_financials_final['Ticker'].isin(stocks_tickers)).all()
print("Passed ticker validation")

assert not df_stock_financials_final.isna().any().any()
print("Passed null validation")

try:
    assert not df_stock_financials_final.duplicated(['Ticker', 'Year', 'Quarter', 'Indicator']).any()
    print("Passed uniqueness validation")
except Exception as e:
    print("Failed uniqueness... Dropping duplicates")
    df_stock_financials_final.drop_duplicates(subset=['Ticker', 'Year', 'Quarter', 'Indicator'], keep='first', inplace=True)

assert not df_stock_financials_final.duplicated(['Ticker', 'Year', 'Quarter', 'Indicator']).any()
print("Passed uniqueness validation")


Passed ticker validation
Passed null validation
Failed uniqueness... Dropping duplicates
Passed uniqueness validation


### Stock News

In [24]:
assert (df_stock_news_final['Ticker'].isin(stocks_tickers)).all()
print("Passed ticker validation")

assert not df_stock_news_final.isna().any().any()
print("Passed null validation")

assert not df_stock_news_final.duplicated(['Index']).any()
print("Passed uniqueness validation")

Passed ticker validation
Passed null validation
Passed uniqueness validation


# 5. Export files
Finally, with the final dataframes for each table, we simply save them to `csv`
files to be imported into the _MySQL_ database.

In [25]:
print("Exporting 'Securities.csv'...")
df_securities_final.to_csv('Securities.csv', index=False)

print("Exporting 'SecurityPrices.csv'...")
df_security_prices_final.to_csv('SecurityPrices.csv', index=False)

print("Exporting 'SecurityInfo.csv'...")
df_stock_info_final.to_csv('SecurityInfo.csv', index=False)

print("Exporting 'StockFinancials.csv'...")
df_stock_financials_final.to_csv('StockFinancials.csv', index=False)

print("Exporting 'FinancialNews.csv'...")
df_stock_news_final.to_csv('FinancialNews.csv', index=False)

Exporting 'Securities.csv'...
Exporting 'SecurityPrices.csv'...
Exporting 'SecurityInfo.csv'...
Exporting 'StockFinancials.csv'...
Exporting 'FinancialNews.csv'...
